# Relative Strength Index (RSI) Indicator

**Formula**: RSI = 100 - (100 / (1 + RS)), where RS = Average Gain / Average Loss

**Interpretation:**
- RSI > 70: Overbought (potential reversal down)
- RSI < 30: Oversold (potential reversal up)
- RSI = 50: Neutral (momentum shift point)
- RSI crossing 50: Momentum change (bullish if crossing up, bearish if crossing down)

**Key Signals:**
- Mean reversion: Buy when RSI < 30, sell when RSI > 70
- Divergence: Price makes new high but RSI doesn't (bearish sign)
- Trend confirmation: RSI staying above 50 in uptrend, below 50 in downtrend
- Trend exhaustion: RSI reaches extreme levels (>90 or <10)

In [1]:
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from indikator import rsi

plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(42)

## Scenario 1: Overbought/Oversold Signals with Price Reversal

In [2]:
# Create data with overbought/oversold conditions and reversals
n = 80
dates = pd.date_range('2024-01-01 09:30', periods=n, freq='5min')

# Build price with strong up move, then reversal from overbought
consolidation = 100 + np.random.randn(15) * 0.4
strong_up = consolidation[-1] + np.cumsum(np.ones(20) * 0.8 + np.random.randn(20) * 0.3)  # Strong uptrend
reversal_down = strong_up[-1] + np.cumsum(-np.ones(25) * 0.5 + np.random.randn(25) * 0.3)  # Reversal
recovery = reversal_down[-1] + np.cumsum(np.ones(20) * 0.4 + np.random.randn(20) * 0.3)  # Recovery

closes = np.concatenate([consolidation, strong_up, reversal_down, recovery])

# Create OHLC
opens = closes + np.random.randn(n) * 0.2
highs = np.maximum(opens, closes) + np.abs(np.random.randn(n)) * 0.4
lows = np.minimum(opens, closes) - np.abs(np.random.randn(n)) * 0.4
volumes = 1000 + np.abs(np.random.randn(n)) * 250

df = pd.DataFrame({'open': opens, 'high': highs, 'low': lows, 'close': closes, 'volume': volumes}, index=dates)
result = rsi(df, window=14)

# Plot
fig = plt.figure(figsize=(16, 11))
gs = GridSpec(3, 1, height_ratios=[2.5, 1.5, 2], hspace=0.3)
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1], sharex=ax1)
ax3 = fig.add_subplot(gs[2], sharex=ax1)

# Candlesticks
for i in range(len(df)):
    color = 'green' if df['close'].iloc[i] >= df['open'].iloc[i] else 'red'
    ax1.plot([i, i], [df['low'].iloc[i], df['high'].iloc[i]], color=color, linewidth=1, alpha=0.8)
    height = abs(df['close'].iloc[i] - df['open'].iloc[i])
    if height < 0.05: height = 0.15
    bottom = min(df['open'].iloc[i], df['close'].iloc[i])
    ax1.add_patch(plt.Rectangle((i-0.3, bottom), 0.6, height, facecolor=color, edgecolor='black', linewidth=0.5, alpha=0.8))

ax1.axvspan(0, 15, alpha=0.15, color='gray', label='Consolidation')
ax1.axvspan(15, 35, alpha=0.15, color='green', label='Strong Uptrend')
ax1.axvspan(35, 60, alpha=0.15, color='red', label='Reversal')
ax1.axvspan(60, 80, alpha=0.15, color='blue', label='Recovery')
ax1.set_ylabel('Price', fontsize=12, fontweight='bold')
ax1.set_title('Scenario 1: Overbought/Oversold Signals with Price Reversal', fontsize=14, fontweight='bold', pad=20)
ax1.legend(loc='upper left', ncol=4)
ax1.grid(True, alpha=0.3)
ax1.set_xlim(-1, len(df))

# Volume
colors_vol = ['green' if df['close'].iloc[i] >= df['open'].iloc[i] else 'red' for i in range(len(df))]
ax2.bar(range(len(df)), volumes, color=colors_vol, alpha=0.7, edgecolor='black', linewidth=0.5)
ax2.set_ylabel('Volume', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.set_xlim(-1, len(df))

# RSI
rsi_vals = result.values
colors_rsi = ['green' if r < 30 else 'red' if r > 70 else 'gray' for r in rsi_vals]
ax3.plot(range(len(rsi_vals)), rsi_vals, 'purple', linewidth=2.5, label='RSI(14)')
ax3.axhline(50, color='black', linestyle='-', linewidth=0.8, alpha=0.5)
ax3.axhline(70, color='red', linestyle='--', linewidth=2.5, label='Overbought (70)', alpha=0.8)
ax3.axhline(30, color='green', linestyle='--', linewidth=2.5, label='Oversold (30)', alpha=0.8)
ax3.fill_between(range(len(df)), 70, 100, alpha=0.2, color='red', label='Overbought Zone')
ax3.fill_between(range(len(df)), 0, 30, alpha=0.2, color='green', label='Oversold Zone')

# Mark overbought and oversold conditions
overbought_idx = np.where(rsi_vals > 70)[0]
oversold_idx = np.where(rsi_vals < 30)[0]
if len(overbought_idx) > 0:
    ax3.scatter(overbought_idx, rsi_vals[overbought_idx], color='red', s=100, marker='o', 
               edgecolor='black', linewidth=1.5, zorder=5, alpha=0.8)
if len(oversold_idx) > 0:
    ax3.scatter(oversold_idx, rsi_vals[oversold_idx], color='green', s=100, marker='o', 
               edgecolor='black', linewidth=1.5, zorder=5, alpha=0.8)

ax3.set_xlabel('Bar Index', fontsize=12, fontweight='bold')
ax3.set_ylabel('RSI', fontsize=12, fontweight='bold')
ax3.set_ylim(0, 100)
ax3.legend(loc='upper right', fontsize=10)
ax3.grid(True, alpha=0.3)
ax3.set_xlim(-1, len(df))

plt.tight_layout()
plt.show()

# Key metrics
max_rsi = result.max()
min_rsi = result.min()
max_idx = result.idxmax()
min_idx = result.idxmin()
max_idx_int = list(df.index).index(max_idx)
min_idx_int = list(df.index).index(min_idx)

print(f"Maximum RSI: {max_rsi:.2f} at bar {max_idx_int} (Overbought)")
print(f"Minimum RSI: {min_rsi:.2f} at bar {min_idx_int} (Oversold)")
print(f"\nPrice at max RSI: ${df['close'].iloc[max_idx_int]:.2f}")
print(f"Price at min RSI: ${df['close'].iloc[min_idx_int]:.2f}")
print(f"\nOverbought bars (RSI > 70): {(result > 70).sum()}")
print(f"Oversold bars (RSI < 30): {(result < 30).sum()}")
print(f"\nThe strong uptrend drives RSI above 70 (overbought).")
print(f"Price reversal occurs when RSI reaches extreme levels.")

TypeError: Type mismatch for parameter 'data' in 'rsi': expected Series, got DataFrame

## Scenario 2: Divergence Detection (Price Strength Divergence)

In [ ]:
# Create price with lower lows but RSI makes higher lows (bullish divergence)
# and higher highs but RSI doesn't reach previous high (bearish divergence)
n2 = 70
dates2 = pd.date_range('2024-01-01 10:00', periods=n2, freq='5min')

# First up move
phase1 = 100 + np.cumsum(np.ones(15) * 0.6 + np.random.randn(15) * 0.3)
# Pullback to lower low
phase2 = phase1[-1] + np.cumsum(-np.ones(12) * 0.5 + np.random.randn(12) * 0.3)
# Second up move to new high
phase3 = phase2[-1] + np.cumsum(np.ones(18) * 0.6 + np.random.randn(18) * 0.3)
# Pullback to lower low again
phase4 = phase3[-1] + np.cumsum(-np.ones(12) * 0.5 + np.random.randn(12) * 0.3)
# Third up move (weaker)
phase5 = phase4[-1] + np.cumsum(np.ones(13) * 0.4 + np.random.randn(13) * 0.3)

closes2 = np.concatenate([phase1, phase2, phase3, phase4, phase5])

# Create OHLC
opens2 = closes2 + np.random.randn(n2) * 0.2
highs2 = np.maximum(opens2, closes2) + np.abs(np.random.randn(n2)) * 0.4
lows2 = np.minimum(opens2, closes2) - np.abs(np.random.randn(n2)) * 0.4
volumes2 = 1000 + np.abs(np.random.randn(n2)) * 250

df2 = pd.DataFrame({'open': opens2, 'high': highs2, 'low': lows2, 'close': closes2, 'volume': volumes2}, index=dates2)
result2 = rsi(df2, window=14)

# Identify price and RSI peaks/troughs for divergence
price_vals = df2['close'].values
rsi_vals2 = result2.values

# Approximate peak detection (simplified)
price_peaks = []
rsi_peaks = []
for i in range(5, len(price_vals) - 5):
    if price_vals[i] > price_vals[i-5] and price_vals[i] > price_vals[i+5] and rsi_vals2[i] > 50:
        price_peaks.append((i, price_vals[i]))
        rsi_peaks.append((i, rsi_vals2[i]))

# Plot
fig = plt.figure(figsize=(16, 11))
gs = GridSpec(3, 1, height_ratios=[2.5, 1.5, 2], hspace=0.3)
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1], sharex=ax1)
ax3 = fig.add_subplot(gs[2], sharex=ax1)

# Candlesticks
for i in range(len(df2)):
    color = 'green' if df2['close'].iloc[i] >= df2['open'].iloc[i] else 'red'
    ax1.plot([i, i], [df2['low'].iloc[i], df2['high'].iloc[i]], color=color, linewidth=1, alpha=0.8)
    height = abs(df2['close'].iloc[i] - df2['open'].iloc[i])
    if height < 0.05: height = 0.15
    bottom = min(df2['open'].iloc[i], df2['close'].iloc[i])
    ax1.add_patch(plt.Rectangle((i-0.3, bottom), 0.6, height, facecolor=color, edgecolor='black', linewidth=0.5, alpha=0.8))

# Mark price peaks
if len(price_peaks) > 0:
    peak_indices = [p[0] for p in price_peaks]
    peak_prices = [p[1] for p in price_peaks]
    ax1.scatter(peak_indices, peak_prices, color='orange', s=200, marker='*', 
               edgecolor='black', linewidth=2, zorder=5, label='Price Peaks')

# Add divergence annotation
ax1.text(0.02, 0.95, 'Bearish Divergence: Price makes new highs but RSI fails to confirm', 
        transform=ax1.transAxes, fontsize=10, verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

ax1.set_ylabel('Price', fontsize=12, fontweight='bold')
ax1.set_title('Scenario 2: Divergence Detection (Price/RSI Divergence)', fontsize=14, fontweight='bold', pad=20)
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)
ax1.set_xlim(-1, len(df2))

# Volume
colors_vol2 = ['green' if df2['close'].iloc[i] >= df2['open'].iloc[i] else 'red' for i in range(len(df2))]
ax2.bar(range(len(df2)), volumes2, color=colors_vol2, alpha=0.7, edgecolor='black', linewidth=0.5)
ax2.set_ylabel('Volume', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.set_xlim(-1, len(df2))

# RSI with divergence zones highlighted
ax3.plot(range(len(rsi_vals2)), rsi_vals2, 'purple', linewidth=2.5, label='RSI(14)')
ax3.axhline(50, color='black', linestyle='-', linewidth=0.8, alpha=0.5)
ax3.axhline(70, color='red', linestyle='--', linewidth=2, label='Overbought (70)', alpha=0.6)
ax3.axhline(30, color='green', linestyle='--', linewidth=2, label='Oversold (30)', alpha=0.6)
ax3.fill_between(range(len(df2)), 70, 100, alpha=0.15, color='red')
ax3.fill_between(range(len(df2)), 0, 30, alpha=0.15, color='green')

# Mark RSI peaks
if len(rsi_peaks) > 0:
    peak_indices = [p[0] for p in rsi_peaks]
    peak_rsis = [p[1] for p in rsi_peaks]
    ax3.scatter(peak_indices, peak_rsis, color='orange', s=200, marker='*', 
               edgecolor='black', linewidth=2, zorder=5, label='RSI Peaks')

# Draw divergence lines
if len(rsi_peaks) >= 2:
    for i in range(len(rsi_peaks) - 1):
        ax3.plot([rsi_peaks[i][0], rsi_peaks[i+1][0]], 
                [rsi_peaks[i][1], rsi_peaks[i+1][1]], 
                'orange', linestyle=':', linewidth=2, alpha=0.7)

ax3.set_xlabel('Bar Index', fontsize=12, fontweight='bold')
ax3.set_ylabel('RSI', fontsize=12, fontweight='bold')
ax3.set_ylim(0, 100)
ax3.legend(loc='upper right')
ax3.grid(True, alpha=0.3)
ax3.set_xlim(-1, len(df2))

plt.tight_layout()
plt.show()

# Analysis
print(f"Price range: ${price_vals.min():.2f} - ${price_vals.max():.2f}")
print(f"Price trend: Higher highs and higher lows (uptrend)")
print(f"\nRSI range: {rsi_vals2.min():.2f} - {rsi_vals2.max():.2f}")
print(f"RSI trend: Peaks getting lower (momentum weakening)")
print(f"\nDIVERGENCE SIGNAL: Bearish divergence detected!")
print(f"Price makes new highs but RSI fails to confirm.")
print(f"This suggests uptrend is weakening and reversal may be coming.")

## Scenario 3: Window Parameter Effect and RSI Sensitivity

In [ ]:
# Create trending data with volatility
n3 = 100
dates3 = pd.date_range('2024-01-01 11:00', periods=n3, freq='5min')

trend = np.linspace(100, 125, n3)
noise = np.random.randn(n3) * 2
closes3 = trend + noise

# Create OHLC
opens3 = closes3 + np.random.randn(n3) * 0.3
highs3 = np.maximum(opens3, closes3) + np.abs(np.random.randn(n3)) * 0.5
lows3 = np.minimum(opens3, closes3) - np.abs(np.random.randn(n3)) * 0.5
volumes3 = 1000 + np.abs(np.random.randn(n3)) * 250

df3 = pd.DataFrame({'open': opens3, 'high': highs3, 'low': lows3, 'close': closes3, 'volume': volumes3}, index=dates3)

# Calculate RSI with different windows
result_w7 = rsi(df3, window=7)
result_w14 = rsi(df3, window=14)
result_w28 = rsi(df3, window=28)

# Plot
fig = plt.figure(figsize=(16, 12))
gs = GridSpec(4, 1, height_ratios=[3, 1.5, 1.5, 1.5], hspace=0.25)

ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1], sharex=ax1)
ax3 = fig.add_subplot(gs[2], sharex=ax1)
ax4 = fig.add_subplot(gs[3], sharex=ax1)

# Candlesticks
for i in range(len(df3)):
    color = 'green' if df3['close'].iloc[i] >= df3['open'].iloc[i] else 'red'
    ax1.plot([i, i], [df3['low'].iloc[i], df3['high'].iloc[i]], color=color, linewidth=0.8, alpha=0.8)
    height = abs(df3['close'].iloc[i] - df3['open'].iloc[i])
    if height < 0.05: height = 0.2
    bottom = min(df3['open'].iloc[i], df3['close'].iloc[i])
    ax1.add_patch(plt.Rectangle((i-0.3, bottom), 0.6, height, facecolor=color, edgecolor='black', linewidth=0.3, alpha=0.8))

ax1.set_ylabel('Price', fontsize=12, fontweight='bold')
ax1.set_title('Effect of Window Parameter on RSI Sensitivity', fontsize=14, fontweight='bold', pad=20)
ax1.grid(True, alpha=0.3)
ax1.set_xlim(-1, len(df3))

# Window = 7 (More sensitive)
rsi_w7 = result_w7.values
ax2.plot(range(len(rsi_w7)), rsi_w7, 'green', linewidth=2, label='Window=7 (More Sensitive)')
ax2.axhline(50, color='black', linestyle='-', linewidth=0.8, alpha=0.3)
ax2.axhline(70, color='red', linestyle='--', linewidth=1.5, alpha=0.5)
ax2.axhline(30, color='green', linestyle='--', linewidth=1.5, alpha=0.5)
ax2.fill_between(range(len(df3)), 70, 100, alpha=0.1, color='red')
ax2.fill_between(range(len(df3)), 0, 30, alpha=0.1, color='green')
ax2.set_ylabel('RSI', fontsize=11, fontweight='bold')
ax2.set_ylim(0, 100)
ax2.grid(True, alpha=0.3)
ax2.legend(loc='upper right')
ax2.set_xlim(-1, len(df3))

# Window = 14 (Standard - Balanced)
rsi_w14 = result_w14.values
ax3.plot(range(len(rsi_w14)), rsi_w14, 'orange', linewidth=2, label='Window=14 (Balanced - Wilder\'s Original)')
ax3.axhline(50, color='black', linestyle='-', linewidth=0.8, alpha=0.3)
ax3.axhline(70, color='red', linestyle='--', linewidth=1.5, alpha=0.5)
ax3.axhline(30, color='green', linestyle='--', linewidth=1.5, alpha=0.5)
ax3.fill_between(range(len(df3)), 70, 100, alpha=0.1, color='red')
ax3.fill_between(range(len(df3)), 0, 30, alpha=0.1, color='green')
ax3.set_ylabel('RSI', fontsize=11, fontweight='bold')
ax3.set_ylim(0, 100)
ax3.grid(True, alpha=0.3)
ax3.legend(loc='upper right')
ax3.set_xlim(-1, len(df3))

# Window = 28 (Smooth - Less sensitive)
rsi_w28 = result_w28.values
ax4.plot(range(len(rsi_w28)), rsi_w28, 'purple', linewidth=2.5, label='Window=28 (Smooth - Less Sensitive)')
ax4.axhline(50, color='black', linestyle='-', linewidth=0.8, alpha=0.3)
ax4.axhline(70, color='red', linestyle='--', linewidth=1.5, alpha=0.5)
ax4.axhline(30, color='green', linestyle='--', linewidth=1.5, alpha=0.5)
ax4.fill_between(range(len(df3)), 70, 100, alpha=0.1, color='red')
ax4.fill_between(range(len(df3)), 0, 30, alpha=0.1, color='green')
ax4.set_xlabel('Bar Index', fontsize=12, fontweight='bold')
ax4.set_ylabel('RSI', fontsize=11, fontweight='bold')
ax4.set_ylim(0, 100)
ax4.grid(True, alpha=0.3)
ax4.legend(loc='upper right')
ax4.set_xlim(-1, len(df3))

plt.tight_layout()
plt.show()

# Statistics
print('RSI Sensitivity by Window:')
print(f'\nWindow = 7 (More Sensitive to Price Changes):')
print(f'  Mean: {rsi_w7.mean():.2f}')
print(f'  Std Dev: {rsi_w7.std():.2f}')
print(f'  Overbought (>70) signals: {(rsi_w7 > 70).sum()}')
print(f'  Oversold (<30) signals: {(rsi_w7 < 30).sum()}')

print(f'\nWindow = 14 (Balanced - Wilder\'s Original):')
print(f'  Mean: {rsi_w14.mean():.2f}')
print(f'  Std Dev: {rsi_w14.std():.2f}')
print(f'  Overbought (>70) signals: {(rsi_w14 > 70).sum()}')
print(f'  Oversold (<30) signals: {(rsi_w14 < 30).sum()}')

print(f'\nWindow = 28 (Smooth - Less Sensitive):')
print(f'  Mean: {rsi_w28.mean():.2f}')
print(f'  Std Dev: {rsi_w28.std():.2f}')
print(f'  Overbought (>70) signals: {(rsi_w28 > 70).sum()}')
print(f'  Oversold (<30) signals: {(rsi_w28 < 30).sum()}')

print(f'\nSmaller windows = more signals (noisier, more false positives)')
print(f'Larger windows = fewer signals (smoother, better trend confirmation)')

## Key Takeaways

**RSI Thresholds & Interpretation:**
- **RSI > 70**: Overbought (potential sell signal, mean reversion opportunity)
- **RSI > 80**: Very overbought (extreme condition, exhaustion warning)
- **RSI 50-70**: Bullish zone (trend confirmation, buy signals within uptrend)
- **RSI 30-50**: Bearish zone (trend confirmation, sell signals within downtrend)
- **RSI < 30**: Oversold (potential buy signal, mean reversion opportunity)
- **RSI < 20**: Very oversold (extreme condition, potential reversal)

**Window Selection:**
- **Small window (5-7)**: More sensitive, faster signals, more false positives
- **Standard window (14)**: Wilder's original, balanced sensitivity, most popular
- **Large window (21-28)**: Smoother, fewer signals, better trend confirmation

**Trading Rules:**
1. **Mean Reversion**: Buy when RSI < 30, sell when RSI > 70 (countertrend)
2. **Trend Confirmation**: Use RSI > 50 to confirm uptrend, RSI < 50 for downtrend
3. **Divergence Trading**: Look for price/RSI divergences (early warning signal)
4. **Extreme Signals**: RSI > 90 or < 10 often precedes sharp reversals
5. **Momentum Shift**: RSI crossing 50 indicates momentum change (bull->bear or vice versa)

**Best Practices:**
- Combine RSI with trend indicators (slope, moving averages)
- Don't fade strong trends: RSI can stay >70 in uptrend, <30 in downtrend
- Use RSI primarily for mean reversion in range-bound markets
- Watch for divergences as early warning of trend exhaustion
- Adjust thresholds (60/40 or 80/20) based on market volatility
- Confirm signals with volume or other indicators

**Window Selection Guide:**
- **Intraday Trading (1-5 min)**: Window 7-9 (faster signals)
- **Swing Trading (5-30 min)**: Window 14 (standard, recommended)
- **Daily Trading**: Window 14-21 (more reliable signals)
- **Longer timeframes**: Window 21-28 (smoother, better trend

**Key Formula Reminder:**
- RSI = 100 - (100 / (1 + RS))
- RS = Average Gain / Average Loss
- Uses Wilder's smoothing method for averaging
- Range: 0-100 (no upper/lower bounds like some indicators)

## Feature Showcase: Safety & Configuration

In [ ]:
# Configuration & Validation Showcase
# ---------------------------------------------------------
import pandas as pd
import numpy as np
from indikator import rsi

# 1. Configuration with .Config()
print(f"--- Custom rsi Configuration ---")
# Create a fully configured version of the indicator
# This factory pattern validates parameters at creation time
custom_rsi = rsi.Config(window=21).make()
print(f"Created: {custom_rsi}")

# 2. Validation Safety
print(f"--- Input Validation Safety ---")
try:
    # Attempt to use invalid data (infinite values)
    invalid_data = pd.Series([100.0, float('inf'), 102.0]) # Infinite values
    print("Attempting calculation with invalid inputs...")
    rsi(invalid_data)
except Exception as e:
    # The @validated decorator automatically catches the issue
    print(f"✓ Validator caught error as expected: {e}")
